In [1]:
import pandas as pd
import torch
from torch.utils.data import DataLoader ,Dataset
from transformers import AutoTokenizer,BertForQuestionAnswering,AutoModel

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [2]:
from transformers import AutoTokenizer,BertForQuestionAnswering,AutoModel
model_checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [3]:
data=pd.read_csv('/kaggle/input/question-pairs-dataset/questions.csv')

In [4]:
questions = ["What is the capital of France?", "Who wrote the play 'Romeo and Juliet'?", "When was the Eiffel Tower built?"]
answers = ["The capital of France is Paris.", "The play 'Romeo and Juliet' was written by William Shakespeare.", "The Eiffel Tower was built in 1889."]

In [5]:
from transformers import DataCollatorWithPadding

In [6]:
w=dict()

In [7]:
data.dropna(inplace=True)

In [8]:
class dataset(Dataset):
    def __init__(self,x,y,z):
        self.x=list(x)
        self.y=list(y)
        out=  tokenizer(self.x,self.y, truncation=True, max_length=30,padding='max_length', return_tensors="pt")
        self.keys=out.keys
        self.out= list(out.items())

        self.z=z
    def __getitem__ (self,idx):
        r={key:value[idx] for key ,value in self.out }

        return r,self.z[idx]
    def __len__(self):
        return len(self.x)

In [9]:
torch.set_default_device('cuda')


In [10]:
data=data.reset_index()

In [11]:
w=dataset(data.iloc[:,-2],data.iloc[:,-3],data.iloc[:,-1])
train=DataLoader(w,batch_size=32,shuffle=True, generator=torch.Generator(device='cuda'))

In [12]:
from transformers import BertTokenizer, BertModel


In [138]:
class bert_compare(torch.nn.Module):
    def __init__ (self):
        super(bert_compare,self).__init__()
        self.bert=BertModel.from_pretrained("bert-base-uncased")
        
        self.Linear=torch.nn.Linear(768,30 )
        self.elu=torch.nn.ELU()
        self.Linear2=torch.nn.Linear(280 ,1 )
        self.cnn1=torch.nn.Conv1d(768,256,kernel_size=2)
        self.cnn2=torch.nn.Conv1d(256,10,kernel_size=2)

        self.relu=torch.nn.ReLU()
    def forward(self,x):
        x=self.bert(**x).last_hidden_state
        x=x.permute(0,2,1)
        x=self.cnn1(x)
        x=self.relu(x)
        x=self.cnn2(x)
        x=torch.nn.Flatten()(x)
        x=self.Linear2(x)
        return x

In [139]:
model=bert_compare()
optim=torch.optim.AdamW(model.parameters(),lr=5e-5)
loss=torch.nn.BCEWithLogitsLoss()


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [140]:
def tok(x,y):
        out=tokenizer(x,y, truncation=True, max_length=30,padding='max_length', return_tensors="pt")
        out={key:value for key,value in out.items()}
        return out
h=tok('my name is mohamed','what is your name')    
model(h)

tensor([[-0.0400]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [126]:
from  tqdm import tqdm

In [127]:
model.train()

bert_compare(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affi

In [128]:
model.bert.train()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [ ]:
for i in range(10):
  loop=tqdm(train,leave=True)
  for x,y in loop :
        optim.zero_grad()
        out=model(x)
        l=loss(out,y.unsqueeze(-1).float())
        l.backward()
        optim.step()
        loop.set_description(f'Epoch {i}')
        loop.set_postfix(loss=l.item())
  torch.save(model,f'model{i}.pth')
  torch.save(model,f'model{i}.pt')

Epoch 0:   5%|▌         | 647/12636 [02:06<40:10,  4.97it/s, loss=0.361]

In [ ]:
tokenizer.tokenize('my name is mohamed ',"where is capital of egypt", truncation=True, max_length=30,padding='max_length', return_tensors="pt")

In [ ]:
word=['my name is mohamed ', "How do I read and find my YouTube comments?" ,"How can I see all my Youtube comments?","How can Internet speed be increased by hacking through DNS?","What is the step by step guide to invest in share market in india?","where is capital of egypt?",'when did you born ','what is your name',"what is capital of egypt",'how old are you']

In [42]:
def tok(x,y):
        out=tokenizer(x,y, truncation=True, max_length=30,padding='max_length', return_tensors="pt")
        out={key:value for key,value in out.items()}
        return out
for i in range(9):
        r=torch.randint(len(word),size=(1,))
        r2=torch.randint(len(word),size=(1,))
        h=tok(word[r],word[r2])    
        e=model(h)
        ans= 'the same' if  int(torch.sigmoid( e)>=.5) else 'not the same'
        print (f'{word[r]} is {ans} {word[r2]}'  )

how old are you is not the same my name is mohamed 
what is your name is not the same How can Internet speed be increased by hacking through DNS?
what is your name is not the same How do I read and find my YouTube comments?
my name is mohamed  is not the same my name is mohamed 
How do I read and find my YouTube comments? is not the same where is capital of egypt?
how old are you is not the same How do I read and find my YouTube comments?
How do I read and find my YouTube comments? is not the same What is the step by step guide to invest in share market in india?
How can I see all my Youtube comments? is the same How can I see all my Youtube comments?
my name is mohamed  is not the same how old are you
